In [1]:
from lusidtools.jupyter_tools import toggle_code

"""FundingLeg - Valuation with daily resets

Attributes
----------
FundingLeg
Compounding
recipes
valuations
"""

toggle_code("Toggle Docstring")

## Table of Contents
* [1. Setup](#setup)
    * [1.1 Create FundingLeg Instrument](#create-fundingleg-instrument)
    * [1.2 Setting up Market Data](#setting-up-market-data)
    * [1.3 Create Portfolio](#create-portfolio)
    * [1.4 Book Transactions](#book-transactions)
* [2. Valuation](#valuation)
    * [2.1 Configure the valuation recipe](#configure-the-valuation-recipe)
    * [2.2 Aggregation](#aggregation)

# Computing FundingLeg PV with daily resets and monthly payment
In this Notebook, we demonstrate how to compute monthly interest amounts using daily ESTER rates.
Monthly payments should be equal to the sum of daily interest amounts computed from ESTER quotes.
Discounting cash flows is not applicable in this example as we are using a recipe with "ConstantTimeValueOfMoney".

In [2]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
import json
import pytz
from dateutil.parser import parse
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as lm


# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_portfolios_response,
    format_quotes_response,
)
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.4f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = lusid.utilities.ApiClientFactory(
        token=RefreshingToken(),
        api_secrets_filename = secrets_path,
        app_name="LusidJupyterNotebook")

In [3]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)
aggregation_api = api_factory.build(lusid.api.AggregationApi)

# 1. Setup <a class="anchor" id="setup"></a>

## 1.1 Create FundingLeg Instrument <a class="anchor" id="create-fundingleg-instrument"></a>

We begin by defining the details of the funding leg with the following characteristics:

- Inception Date: 2022-01-02
- Maturity Date: 2030-12-12
- Currency: EUR
- Initial Notional: 0
- Index: EUR.ESTER.1D
- Spread: 50 bp


Notice that the initial notional is set to zero, as we will be booking this later on in the transactions. For accruals, the funding leg will store the notional history and accrue accordingly.

In [4]:
# Set swap details for start date and maturity
start_date = datetime(2022, 1, 1, 0, 0, tzinfo=pytz.utc)

effective_delta = 1

# Funding leg specifications
market_data_scope = "FBN"
effective_at = start_date
end_date = datetime(2030, 12, 12, 0, 0, tzinfo=pytz.utc)
currency = "EUR"
initial_notional = 0
spread = 0.005 # 50bp
leg_identifier="FundingLeg001"
identifier_type = "ClientInternal"
leg_name = "Funding Leg EUR.ESTER.1D.ESTER"

# The market data supplier
market_supplier = "Lusid"

In [5]:
# Define the flow conventions of the floating leg
floating_leg_convention = lm.FlowConventions(
    currency=currency,
    payment_frequency="1M",
    day_count_convention="Act360",
    roll_convention="ModifiedFollowing",
    payment_calendars=[],
    reset_calendars=[],
    settle_days=0,
    reset_days=0
)

# Define the flow conventions of the underlying index
floating_leg_idx_convention = lm.IndexConvention(
    currency=currency,
    code="ESTER",
    payment_tenor="1D",
    fixing_reference="EUR.ESTER.1D.ESTER",
    publication_day_lag=0,
    day_count_convention="Act360"
)

compounding_1d = lm.Compounding(
    calculation_shift_method = "NoShift",
    compounding_method = "None",
    reset_frequency = "1D",
    spread_compounding_method = "None",
)

# Set the leg definition and side
floating_leg_definition = lm.LegDefinition(
    conventions=floating_leg_convention,
    index_convention = floating_leg_idx_convention,
    notional_exchange_type = "None",
    pay_receive = "Pay",
    rate_or_spread = spread,
    stub_type = "Back",
    compounding = compounding_1d
)

# Define the funding leg
funding_leg = lm.FundingLeg(
    start_date=start_date,
    maturity_date=end_date,
    notional=initial_notional,
    leg_definition=floating_leg_definition,
    instrument_type="FundingLeg"
)

# Upsert the instrument to LUSID
instrument_definition = lusid.models.InstrumentDefinition(
    name=leg_name,
    identifiers={f"{identifier_type}": lusid.models.InstrumentIdValue(f"{leg_identifier}")},
    definition=funding_leg
)
upsert_request = {f"{leg_identifier}": instrument_definition}

# Upserts the instrument to LUSID
response = instruments_api.upsert_instruments(scope=market_data_scope,
    request_body=upsert_request)

if response.failed == {}:
    print(f"Instrument {instrument_definition.name} was successfully upserted into LUSID")
    print(f"Instrument created with LUID:{response.values[leg_identifier].lusid_instrument_id}")
else:
    print("An error occurred with the above upsert_instruments call, see error message:", response.failed)



Instrument Funding Leg EUR.ESTER.1D.ESTER was successfully upserted into LUSID
Instrument created with LUID:LUID_0004E3SH


## 1.2 Setting up Market Data <a class="anchor" id="setting-up-market-data"></a>

We load ESTER rates into lusid, from 2022-01-02 to 2022-03-02.

In [6]:
# Read quotes and adjust date objects
df = pd.read_csv("data/ester_rates.csv")
df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)
df.head()

,Date,Price,Ticker
0,2022-01-01,0.5000,EUR.ESTER.1D.ESTER
1,2022-01-02,0.5000,EUR.ESTER.1D.ESTER
2,2022-01-03,0.5000,EUR.ESTER.1D.ESTER
3,2022-01-04,0.5000,EUR.ESTER.1D.ESTER
4,2022-01-05,0.5000,EUR.ESTER.1D.ESTER


In [7]:
# Create a quotes mapping for Lusid Python Tools
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$RIC",
    "quote_id.effective_at": "Date",
    "quote_id.quote_series_id.field": "$mid",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Rate",
    "quote_id.quote_series_id.instrument_id": "Ticker",
    "metric_value.value": "Price",
    "metric_value.unit": "$EUR",
    "scale_factor":100
}

result = load_from_data_frame(
    api_factory = api_factory,
    scope=market_data_scope,
    data_frame=df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes"
)

succ, failed, errors = format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

,success,failed,errors
0,61,0,0


## 1.3 Create Portfolio <a class="anchor" id="create-portfolio"></a>

In [8]:
portfolio_code = "FundingLeg_Portfolio_EUR"

try:
    portfolio = transaction_portfolio_api.create_portfolio(
        scope=market_data_scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            instrument_scopes=[market_data_scope],
            code=portfolio_code,
            base_currency="EUR",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )
        
    print(f"Portfolio {portfolio.id} was successfully upserted into LUSID")

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'FundingLeg_Portfolio_EUR' because it already exists in scope 'FBN'.


## 1.4 Book Transactions <a class="anchor" id="book-transactions"></a>
With the portfolio in LUSID, we can book a transaction against our FundingLeg "FundingLeg001".

In [9]:
primary_instrument_identifier = { "Instrument/default/ClientInternal": leg_identifier }

upsert_transactions = transaction_portfolio_api.upsert_transactions(
        scope=market_data_scope,
        code=portfolio_code,
        transaction_request=[
            lm.TransactionRequest(
                transaction_id="txn_test",
                type="Buy",
                instrument_identifiers=primary_instrument_identifier,
                transaction_date="2022-01-01",
                settlement_date="2022-01-02",
                units=1000000, # FundingLeg Notional 1'000'000
                transaction_price=lm.TransactionPrice(
                    price=0, type="Price"
                ),
                total_consideration=lm.CurrencyAndAmount(
                    amount=0, currency="EUR"
                ),
                properties={
                    f"Transaction/default/RequiresFundingLegHistory": lm.ModelProperty(
                        key="Transaction/default/RequiresFundingLegHistory",
                        value=lm.PropertyValue(label_value="true"),
                    ),
                },
            )
        ],
    )

# 2. Valuation <a class="anchor" id="valuation"></a>
For interacting with the valuation engine, we will need a configuration recipe, which are a set of steps that define how a valuation is to be carried out. Generally, this is part of LUSID configuration that requires a one time setup, below we will be focusing on the _FundingLeg_ instrument.

## 2.1 Configure the valuation recipe <a class="anchor" id="configure-the-valuation-recipe"></a>
We begin by defining the pricing context under which valuation will be done, this allows to select the model under which we can value the instrument. In our case we have selected _"ConstantTimeValueOfMoney"_, which will use a deterministic pricer to calculate our portfolio PV/Accrual.

In [10]:
# Define instrument model config


model_option = {
    "expectedFundingLegNotional": "Zero",
    "modelOptionsType": "FundingLegModelOptions"
}

def create_pricing_context(market_supplier: str):
    
    model_rules = [
        lm.VendorModelRule(
                supplier=market_supplier,
                model_name="ConstantTimeValueOfMoney",
                instrument_type="FundingLeg",
                parameters="{}",
                model_options = model_option
            )
    ]
    
    return lm.PricingContext(
        model_rules = model_rules,
         options=lm.PricingOptions(
            model_selection=lm.ModelSelection(
                        library=market_supplier,
                        model="SimpleStatic"
                    )
         )
    )

# Create the pricing context for the recipe
pricing_context = create_pricing_context(market_supplier)


In [11]:
def create_market_context(market_data_scope, supplier):
    return lm.MarketContext(
        # Set rules for where we should resolve our rates and equity data.
        # looking for quotes given a list of identifier types (clientInternal, RIC..)
        market_rules=[
            lm.MarketDataKeyRule(
                key='Quote.ClientInternal.*',
                data_scope=market_data_scope,
                supplier=supplier,
                quote_type='Price',
                field='mid',
                quote_interval="2D.0D"),# looking for the most recent quote value within a 2-days window
             lm.MarketDataKeyRule(
                key='Quote.RIC.*',
                data_scope=market_data_scope,
                supplier=supplier,
                quote_type='Rate',
                field='mid',
                quote_interval="2D.0D"),
            lm.MarketDataKeyRule(
                key='Quote.RIC.*',
                data_scope=market_data_scope,
                supplier=supplier,
                quote_type='Price',
                field='mid',
                quote_interval="2D.0D"),

        ],
        # Control default options for resolving market data.
        options=lm.MarketOptions(
            default_supplier=supplier,
            default_instrument_code_type="ClientInternal",
            default_scope=market_data_scope)
     )

    return market_context

# In our case simply default to the LUSID market_supplier.
market_context = create_market_context(market_data_scope, market_supplier)

In [12]:
def create_pricing_recipe(scope, market_context, pricing_context):

    return lm.ConfigurationRecipe(
        scope=market_data_scope,
        code="FundingLeg",
        description="Funding leg pricing recipe",
        market=market_context,
        pricing=pricing_context
    )

# Create the funding leg pricing recipe using our market_data_scope
pricing_recipe = create_pricing_recipe(market_data_scope, market_context, pricing_context)

In [13]:
def upsert_recipe(recipe):
    recipe_request = lm.UpsertRecipeRequest(
        configuration_recipe = recipe
    )
    
    return configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request = recipe_request)

# Upsert the previously created recipe to be used in valuation
response = upsert_recipe(pricing_recipe)
print(f"Recipe upserted at {response.value}")

Recipe upserted at 2022-11-15 15:16:28.506181+00:00


## 2.2 Aggregation <a class="anchor" id="aggregation"></a>
aggregate_pricing() function when given a start date & end date along with a portfolio returns a series of valuations on held FundingLeg. These valuations therefore enable you to know the current unsettled interest amount between and on reset dates.

In [14]:
def aggregate_pricing(portfolio_scope, portfolio_code, recipe_scope, recipe_code, metrics, effective_at, effective_from=None, group_by=[]):
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(
            scope=recipe_scope,
            code=recipe_code
        ),
        metrics=metrics,
            portfolio_entity_ids=[
                lm.PortfolioEntityId(scope=portfolio_scope, code=portfolio_code)
            ],
            valuation_schedule=lm.ValuationSchedule(
                effective_from=effective_from.isoformat(),
                effective_at=effective_at.isoformat()
            ),
        sort=[
            lm.OrderBySpec(key='Analytic/default/ValuationDate',
            sort_order='Ascending')
        ],
        group_by=group_by
        )
    return aggregation_api.get_valuation(
    valuation_request= valuation_request
    )

In [15]:
# Create the metrics list for the desired data
metrics = [
        lm.AggregateSpec(key='Valuation/PV',
                             op='Value'),
        lm.AggregateSpec(key='Instrument/default/Name',
                             op='Value'),
        lm.AggregateSpec(key='Holding/default/Units',
                             op='Value'),
        lm.AggregateSpec(key='Valuation/Accrued',
                             op='Value'),
        lm.AggregateSpec(key='Analytic/default/ValuationDate',
                             op='Value'),
        ]
# Store results
results = aggregate_pricing(market_data_scope, portfolio_code, market_data_scope, "FundingLeg", metrics, start_date + timedelta(days=32), start_date)

In [16]:
def valuation_response_to_df(results: lusid.ListAggregationResponse) -> pd.DataFrame:
    """"
    This Function returns a pandas Data Frame from a LUSID valuation
    response object
    :param lusid.ListAggregationResponse: LUSID response object
    :return: pandas.DataFrame
    """
    headers = set(results.data[0].keys())

    data = {
        header: [result[header] for result in results.data] for header in headers
    }
    return pd.DataFrame(data)

In [17]:
# Display PVs
valuations = valuation_response_to_df(results)
valuations['Analytic/default/ValuationDate'] = valuations['Analytic/default/ValuationDate'].apply(lambda x : parse(x).strftime('%d/%m/%Y'))
valuations.set_index('Analytic/default/ValuationDate', inplace=True)
display(valuations)

,Valuation/PV,Valuation/Accrued,Instrument/default/Name,Holding/default/Units
Analytic/default/ValuationDate,,,,
03/01/2022,0.0000,0.0000,Funding Leg EUR.ESTER.1D.ESTER,"1,000,000.0000"
04/01/2022,-55.5556,-55.5556,Funding Leg EUR.ESTER.1D.ESTER,"1,000,000.0000"
05/01/2022,-111.1111,-111.1111,Funding Leg EUR.ESTER.1D.ESTER,"1,000,000.0000"
06/01/2022,-166.6667,-166.6667,Funding Leg EUR.ESTER.1D.ESTER,"1,000,000.0000"
07/01/2022,-222.2222,-222.2222,Funding Leg EUR.ESTER.1D.ESTER,"1,000,000.0000"
10/01/2022,-388.8889,-388.8889,Funding Leg EUR.ESTER.1D.ESTER,"1,000,000.0000"
11/01/2022,-444.4444,-444.4444,Funding Leg EUR.ESTER.1D.ESTER,"1,000,000.0000"
12/01/2022,-500.0000,-500.0000,Funding Leg EUR.ESTER.1D.ESTER,"1,000,000.0000"
13/01/2022,-555.5556,-555.5556,Funding Leg EUR.ESTER.1D.ESTER,"1,000,000.0000"


The valuation above shows how PV accrues every day until the monthly Leg reset. On each valuation date, the effecive ESTER quote is used to compute interest amounts: Notional * (quote + spread) * (1/360)